In [0]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

In [2]:
tf.__version__

'2.2.0-rc2'

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout,concatenate
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional, GRU
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler

In [6]:
#using google colab mounting drive to colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
tweet = pd.read_csv("/content/drive/My Drive/google_colab/Tweets.csv")
tweet.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [8]:
from nltk.tokenize import RegexpTokenizer
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df
    
data = tweet[['text', 'airline_sentiment']]
tweet_text = standardize_text(data, "text")

tokenizer = RegexpTokenizer(r'\w+')
tweet_text["tokens"] = tweet_text["text"].apply(tokenizer.tokenize)
#transforming postive to 2, netural to 1, negative to 0
sentiment = sorted(tweet_text['airline_sentiment'].unique())
print (sentiment)
sentiment_mapping = dict(zip(sentiment, range(0, len(sentiment) + 1)))
tweet_text['airline_sentiment']  = tweet_text['airline_sentiment'].map(sentiment_mapping).astype(int)

all_words = [word for tokens in tweet_text["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in tweet_text["tokens"]]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max & min sentence length is %s & %s" %(max(sentence_lengths), min(sentence_lengths)))

['negative', 'neutral', 'positive']
247352 words total, with a vocabulary size of 13088
Max & min sentence length is 34 & 1


In [9]:
tweet.shape, data.shape

((14640, 15), (14640, 3))

In [0]:
import gensim
word2vec_path = "/content/drive/My Drive/google_colab/GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [0]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
  if len(tokens_list)<1:
    return np.zeros(k)
  if generate_missing:
    vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
  else:
    vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
  length = len(vectorized)
  summed = np.sum(vectorized, axis=0)
  averaged = np.divide(summed, length)
  return averaged

def get_word2vec_embeddings(vectors, data, generate_missing=False):
  embeddings = data['tokens'].apply(lambda x: get_average_word2vec(x, vectors,generate_missing=generate_missing))
  return list(embeddings)

In [12]:
embeddings = get_word2vec_embeddings(word2vec, data)
len(embeddings)

14640

In [13]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 34
VOCAB_SIZE = len(VOCAB)
VALIDATION_SPLIT=.3
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(data["text"].tolist()) #tokenize
sequences = tokenizer.texts_to_sequences(data["text"].tolist()) #list of each sentence as number index 
word_index = tokenizer.word_index  #index of tokenized  word
print('Found %s unique tokens.' % len(word_index))

cnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #dding padding to front , converiting all sentece length to 34 
labels = to_categorical(np.asarray(tweet_text['airline_sentiment']))
#randomly suffle data
indices = np.arange(cnn_data.shape[0])
np.random.shuffle(indices)
cnn_data = cnn_data[indices]
labels = labels[indices]

embedding_weights = np.zeros((len(word_index)+1, EMBEDDING_DIM))
#extracting each word embedding of 300 size
for word,index in word_index.items():
    embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(embedding_weights.shape)

Found 13346 unique tokens.
(13347, 300)


In [14]:
num_words = len(word_index)+1
# embedding_layer = Embedding(num_words,
#                             EMBEDDING_DIM,
#                             weights=[embedding_weights],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

#input to the model will be embedding of sentence as follow 
#sentence is : "flight is good" 
#this sentence will convert into size of 34 workds with padding
#now each word will have embedding of 300
#so input matrix will be 34x300
train_sample = int(0.7 * cnn_data.shape[0])
val_samples = int(0.15 * cnn_data.shape[0])
test_samples = int(0.15 * cnn_data.shape[0])

x_train = cnn_data[:train_sample]
y_train = labels[:train_sample]
x_val = cnn_data[train_sample:train_sample+val_samples]
y_val = labels[train_sample:train_sample+val_samples]
x_test = cnn_data[-test_samples:]
y_test = labels[-test_samples:]
print (x_train.shape, x_val.shape, x_test.shape, y_val.shape,y_train.shape)
#print (x_train[-5:], x_val[-5:], x_test[-5:])

(10248, 34) (2196, 34) (2196, 34) (2196, 3) (10248, 3)


In [0]:
def LSTM_model(num_words,EMBEDDING_DIM,embedding_weights,max_sequence_length,labels_index):
  model = Sequential()
  model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_weights],
                      input_length=max_sequence_length,
                     trainable=False))
  model.add(LSTM(100,dropout=0.2))
  model.add(Dropout(0.2))
  model.add(Dense(labels_index, activation='sigmoid'))
  print(model.summary())
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                metrics=['acc'])
  return model

In [16]:
model = LSTM_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))
#model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=128)
callback = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback],validation_data=(x_val, y_val))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 34, 300)           4004100   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               160400    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 4,164,803
Trainable params: 160,703
Non-trainable params: 4,004,100
_________________________________________________________________
None
Epoch 1/50
321/321 [==============================] - 2s 8ms/step - loss: 0.6858 - acc: 0.7263 - val_loss: 0.5797 - val_acc: 0.7719
Epoch 2/50
321/321 [==============================] - 2s 6ms/step - loss: 0.5641 - acc: 0.7766 - val_

Model weights extraction 

In [19]:
for e in zip(model.layers[1].trainable_weights, model.layers[1].get_weights()):
  print('Param %s:\n%s' % (e[0],e[1]))

Param <tf.Variable 'lstm/lstm_cell/kernel:0' shape=(300, 400) dtype=float32, numpy=
array([[ 0.08815192,  0.09716977,  0.00254296, ..., -0.09403526,
        -0.01898826,  0.15587874],
       [ 0.15319464, -0.02541624, -0.03523511, ..., -0.08448473,
         0.02594964, -0.07609219],
       [ 0.0238332 , -0.08978818,  0.00697285, ..., -0.12641865,
         0.04266918, -0.100934  ],
       ...,
       [-0.12953196, -0.08304246,  0.01244843, ...,  0.06062976,
         0.0079515 ,  0.08774588],
       [-0.1479794 , -0.02860249, -0.02260115, ...,  0.17241871,
         0.07891668, -0.03710394],
       [ 0.00805102, -0.04112032,  0.0064938 , ...,  0.06257927,
         0.1095733 ,  0.04310419]], dtype=float32)>:
[[ 0.08815192  0.09716977  0.00254296 ... -0.09403526 -0.01898826
   0.15587874]
 [ 0.15319464 -0.02541624 -0.03523511 ... -0.08448473  0.02594964
  -0.07609219]
 [ 0.0238332  -0.08978818  0.00697285 ... -0.12641865  0.04266918
  -0.100934  ]
 ...
 [-0.12953196 -0.08304246  0.01244843 

In [0]:
units = 100
W = model.layers[1].get_weights()[0]
U = model.layers[1].get_weights()[1]
b = model.layers[1].get_weights()[2]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

U_i = U[:, :units]
U_f = U[:, units: units * 2]
U_c = U[:, units * 2: units * 3]
U_o = U[:, units * 3:]

b_i = b[:units]
b_f = b[units: units * 2]
b_c = b[units * 2: units * 3]
b_o = b[units * 3:]

In [24]:
print (W.shape, U.shape, b.shape)
print (W_i.shape, W_f.shape, W_c.shape, W_o.shape)
print (U_i.shape,U_f.shape,U_c.shape,U_o.shape,)

(300, 400) (100, 400) (400,)
(300, 100) (300, 100) (300, 100) (300, 100)
(100, 100) (100, 100) (100, 100) (100, 100)


In [0]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 4ms/step - loss: 0.5102 - acc: 0.7987


[0.510235071182251, 0.7987249493598938]

Tips:

1. Can save model when accuracy improve 
2. Earlystoping can also be used if validation accuracy loss is not decreaing or validation accuracy is not increasing. 


CNN and LSTM based model

In [0]:
def CNN_LSTM_model(num_words,EMBEDDING_DIM,embedding_weights,max_sequence_length,labels_index):
  model = Sequential()
  model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_weights],
                      input_length=max_sequence_length,
                     trainable=False))
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100,dropout=0.2))
  model.add(Dense(labels_index, activation='sigmoid'))
  print(model.summary())
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                metrics=['acc'])
  return model

In [0]:
model = CNN_LSTM_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))
#model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=128)
callback = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback],validation_data=(x_val, y_val))

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 34, 300)           4004100   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 34, 32)            28832     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 17, 32)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 303       
Total params: 4,086,435
Trainable params: 82,335
Non-trainable params: 4,004,100
_________________________________________________________________
None
Epoch 1/50
321/321 [==============================] - 2s 7ms/step - loss: 0.6964 - acc: 0.7148 - va

In [0]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 4ms/step - loss: 0.5046 - acc: 0.8010


[0.5045666694641113, 0.8010018467903137]

GRU based model 

In [0]:
def GRU_model(num_words,EMBEDDING_DIM,embedding_weights,max_sequence_length,labels_index):
  model = Sequential()
  model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_weights],
                      input_length=max_sequence_length,
                     trainable=False))
  model.add(GRU(100,dropout=0.2))
  model.add(Dropout(0.2))
  model.add(Dense(labels_index, activation='sigmoid'))
  print(model.summary())
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                metrics=['acc'])
  return model

In [0]:
model = GRU_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))
#model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=128)
callback = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback],validation_data=(x_val, y_val))

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 34, 300)           4004100   
_________________________________________________________________
gru_3 (GRU)                  (None, 100)               120600    
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 303       
Total params: 4,125,003
Trainable params: 120,903
Non-trainable params: 4,004,100
_________________________________________________________________
None
Epoch 1/50
321/321 [==============================] - 2s 6ms/step - loss: 0.7009 - acc: 0.7129 - val_loss: 0.5401 - val_acc: 0.7901
Epoch 2/50
321/321 [==============================] - 2s 6ms/step - loss: 0.5588 - acc: 0.7734 - v

In [0]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 3ms/step - loss: 0.4914 - acc: 0.8019


[0.49135515093803406, 0.8019125461578369]

Bidirectional based model

In [0]:
def Bidirectional_model(num_words,EMBEDDING_DIM,embedding_weights,max_sequence_length,labels_index):
  model = Sequential()
  model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_weights],
                      input_length=max_sequence_length,
                     trainable=False))
  model.add(Bidirectional(LSTM(100,dropout=0.3)))
  model.add(Dropout(0.3))
  model.add(Dense(100, activation='tanh'))
  model.add(Dense(labels_index, activation='sigmoid'))
  print(model.summary())
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                metrics=['acc'])
  return model

In [0]:
model = Bidirectional_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))
#model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=128)
callback = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback],validation_data=(x_val, y_val))

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 34, 300)           4004100   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               320800    
_________________________________________________________________
dropout_14 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 303       
Total params: 4,345,303
Trainable params: 341,203
Non-trainable params: 4,004,100
_________________________________________________________________
None
Epoch 1/50
321/321 [==============================] - 4s 11ms/step - loss: 0.6849 - acc: 0.7247 - 

In [0]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 5ms/step - loss: 0.4932 - acc: 0.8033


[0.49323299527168274, 0.8032786846160889]

Bidirectional model with LearningRateScheduler

In [0]:
model = Bidirectional_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))

def scheduler(epoch):
  if epoch < 10:
    return 0.01
  else:
    return 0.0001 * tf.math.exp(0.1 * (10 - epoch))
lr_scheduler = LearningRateScheduler(scheduler)
callback = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback,lr_scheduler],validation_data=(x_val, y_val))

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 34, 300)           4004100   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 200)               320800    
_________________________________________________________________
dropout_20 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 303       
Total params: 4,345,303
Trainable params: 341,203
Non-trainable params: 4,004,100
_________________________________________________________________
None
Epoch 1/50
321/321 [==============================] - 3s 11ms/step - loss: 0.8267 - acc: 0.6640 - 

In [0]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 5ms/step - loss: 0.5010 - acc: 0.7942


[0.5009780526161194, 0.7941712141036987]

Deep LSTM Network 

In [0]:
def Deep_LSTM_model(num_words,EMBEDDING_DIM,embedding_weights,max_sequence_length,labels_index):
  model = Sequential()
  model.add(Embedding(num_words, EMBEDDING_DIM, weights=[embedding_weights],
                      input_length=max_sequence_length,
                     trainable=False))
  model.add(LSTM(100,dropout=0.2,return_sequences=True))
  model.add(LSTM(100,dropout=0.2,return_sequences=True))
  model.add(LSTM(100,dropout=0.2))
  model.add(Dropout(0.2))
  model.add(Dense(labels_index, activation='sigmoid'))
  print(model.summary())
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                metrics=['acc'])
  return model

In [34]:
model = Deep_LSTM_model(num_words, EMBEDDING_DIM,embedding_weights,MAX_SEQUENCE_LENGTH,len(list(tweet_text['airline_sentiment'].unique())))

def scheduler(epoch):
  if epoch < 10:
    return 0.01
  else:
    return 0.0001 * tf.math.exp(0.1 * (10 - epoch))
lr_scheduler = LearningRateScheduler(scheduler)
callback = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True,mode="min")
# This callback will stop the training when there is no improvement in
# the validation loss for 5 consecutive epochs.
model.fit(x_train, y_train, epochs=50, callbacks=[callback,lr_scheduler],validation_data=(x_val, y_val))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 34, 300)           4004100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 34, 100)           160400    
_________________________________________________________________
lstm_5 (LSTM)                (None, 34, 100)           80400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 4,325,603
Trainable params: 321,503
Non-trainable params: 4,004,100
______________________________________

In [35]:
model.evaluate(x_test, y_test, batch_size = 128)

18/18 [==============================] - 0s 6ms/step - loss: 0.5292 - acc: 0.8097


[0.5291927456855774, 0.8096539378166199]

Model weights extraction 

In [36]:
for e in zip(model.layers[1].trainable_weights, model.layers[1].get_weights()):
  print('Param %s:\n%s' % (e[0],e[1]))


Param <tf.Variable 'lstm_4/lstm_cell_4/kernel:0' shape=(300, 400) dtype=float32, numpy=
array([[-0.62566406, -0.19284685,  0.06901129, ..., -0.22694892,
        -0.9761474 , -0.18277022],
       [-0.14448401, -0.20136186, -0.08714977, ..., -0.02943275,
         0.22990897, -0.24407145],
       [-0.37502635, -0.38083982,  0.7504096 , ..., -0.24946189,
        -0.18547767, -0.49076247],
       ...,
       [ 0.39293703,  0.27259287,  0.08014014, ...,  0.07944054,
         0.32485613, -0.23155512],
       [-0.3665875 , -0.2076555 , -0.06240206, ..., -0.00664771,
        -0.0648846 , -0.01105047],
       [ 0.38626614, -0.07425579, -0.2517469 , ...,  0.10659853,
        -0.249037  , -0.07017116]], dtype=float32)>:
[[-0.62566406 -0.19284685  0.06901129 ... -0.22694892 -0.9761474
  -0.18277022]
 [-0.14448401 -0.20136186 -0.08714977 ... -0.02943275  0.22990897
  -0.24407145]
 [-0.37502635 -0.38083982  0.7504096  ... -0.24946189 -0.18547767
  -0.49076247]
 ...
 [ 0.39293703  0.27259287  0.080140

In [43]:
units = 100
layers = [1,2,3]
for i in layers:
  W = model.layers[i].get_weights()[0]
  U = model.layers[i].get_weights()[1]
  b = model.layers[i].get_weights()[2]

  W_i = W[:, :units]
  W_f = W[:, units: units * 2]
  W_c = W[:, units * 2: units * 3]
  W_o = W[:, units * 3:]

  U_i = U[:, :units]
  U_f = U[:, units: units * 2]
  U_c = U[:, units * 2: units * 3]
  U_o = U[:, units * 3:]

  b_i = b[:units]
  b_f = b[units: units * 2]
  b_c = b[units * 2: units * 3]
  b_o = b[units * 3:]
  print("\nlstm layer " + str(i) + " shapes")
  print (W.shape, U.shape, b.shape)
  print (W_i.shape, W_f.shape, W_c.shape, W_o.shape)
  print (U_i.shape,U_f.shape,U_c.shape,U_o.shape)
  print (b_i.shape,b_f.shape,b_c.shape,b_o.shape)


lstm layer 1 shapes
(300, 400) (100, 400) (400,)
(300, 100) (300, 100) (300, 100) (300, 100)
(100, 100) (100, 100) (100, 100) (100, 100)
(100,) (100,) (100,) (100,)

lstm layer 2 shapes
(100, 400) (100, 400) (400,)
(100, 100) (100, 100) (100, 100) (100, 100)
(100, 100) (100, 100) (100, 100) (100, 100)
(100,) (100,) (100,) (100,)

lstm layer 3 shapes
(100, 400) (100, 400) (400,)
(100, 100) (100, 100) (100, 100) (100, 100)
(100, 100) (100, 100) (100, 100) (100, 100)
(100,) (100,) (100,) (100,)
